In [89]:
# declares packages
import os, re, shutil, json
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sec_edgar_downloader import Downloader
import xmltodict
import numpy as np

In [153]:
# downloads forms from sec
dl = Downloader("")
form = "4"
ticker= "SCHW"
year= "2022"
month= "05"
day= "01"
# TODO: calculate dates missing based on files saved locally
dl.get(form, ticker, after=year+'-'+month+'-'+day)

17

In [154]:
# making directories
path_form = os.path.join("sec-edgar-filings", ticker, form)
if not os.path.isdir(path_form):
    print("Error: path is not a directory")  

path_json = os.path.join("form4-json")
os.makedirs(path_json, exist_ok = True)

# converting to json
aggregates = {}
pattern = re.compile("([0-9]+)")
for filingDir in os.listdir(path_form):
    fullSubmissionFname = os.path.join(path_form, filingDir, "filing-details.xml")
    with open(fullSubmissionFname) as xml_file:
        data_dict = xmltodict.parse(xml_file.read())
        xml_file.close()
        data_dict[filingDir] = data_dict.pop('ownershipDocument')
        aggregates = {**aggregates, **data_dict}

json_data = json.dumps(aggregates)
jsonFname = path_json+'/'+ticker+".json"
with open(jsonFname, "w") as json_file:
    json_file.write(json_data)
    json_file.close()

print('Total Form 4 Processed: ' + str(len(aggregates)))


Total Form 4 Processed: 546


In [ ]:
# TODO: return json with only info needed
path = os.path.join("sec-edgar-filings", ticker, form)
pattern = re.compile("([0-9]+)")
for filingDir in os.listdir(path):
    fullSubmissionFname = os.path.join(path, filingDir, "filing-details.xml")
    with open(fullSubmissionFname) as xml_file:
        data_dict = xmltodict.parse(xml_file.read())
        xml_file.close()      
        data = {
            'periodOfReport': data_dict['ownershipDocument']['periodOfReport'], 
            'reportingOwner': data_dict['ownershipDocument']['reportingOwner']['reportingOwnerRelationship'], 
            'nonDerivativeTransaction': data_dict['ownershipDocument']["nonDerivativeTable"]["nonDerivativeTransaction"]["transactionCoding"]["transactionCode"],
        }
        print(pd.json_normalize(data).info)

In [ ]:
# TODO print(aggregates)
# df = pd.DataFrame.from_dict(aggregates, orient='index')
df = pd.json_normalize(aggregates.pop('0001209191-22-027197'))
print(df.info)

In [ ]:
def get_name(reporting_owner):
    try:
        if type(reporting_owner) is dict:
            print(reporting_owner["reportingOwnerId"]["rptOwnerName"])
        if type(reporting_owner) is list:
            for transaction in reporting_owner:
                print(transaction["reportingOwnerId"]["rptOwnerName"])
    except Exception as exception:
        print(exception, key)# return []

def get_relationship(reporting_owner):
    try:
        if type(reporting_owner) is dict:
            print(reporting_owner["reportingOwnerRelationship"])
        if type(reporting_owner) is list:
            for transaction in reporting_owner:
                print(transaction["reportingOwnerRelationship"])
    except Exception as exception:
        print(exception, key)

def get_transaction_code(transactions):
    try:
        if type(transactions) is list:
            for transaction in transactions:
                print(transaction["transactionCoding"]["transactionCode"])
                print(transaction["transactionAmounts"]["transactionShares"]['value'])
        if type(transactions) is dict:
            print(transactions["transactionCoding"]["transactionCode"])
            print(transactions["transactionAmounts"]["transactionShares"]['value'])            
    except Exception as exception:
        print(exception, key)

for (key, entry) in aggregates.items():
    try:
        names = get_name(entry["reportingOwner"])
        relationship = get_relationship(entry["reportingOwner"])
        code = get_transaction_code(entry["nonDerivativeTable"]["nonDerivativeTransaction"])
        date = entry['periodOfReport']

    except Exception as exception:
            print('func', exception, filingDir, key)


In [165]:
newDict = dict()
# Iterate over all the items in dictionary and filter items which has even keys
for (key, value) in aggregates.items():
    print(key)
    reporting_owner = value["reportingOwner"]
    if (type(reporting_owner) is dict) and reporting_owner["reportingOwnerRelationship"].get("isOfficer", False) == "1":
        print(reporting_owner["reportingOwnerId"]["rptOwnerName"])
        #get code
        if value.get("nonDerivativeTable", False):
            get_transaction_code(value["nonDerivativeTable"]["nonDerivativeTransaction"])      
        # get date
        print(value['periodOfReport'])

        newDict[key] = value

    if type(reporting_owner) is list:
        for transaction in reporting_owner:
            if transaction["reportingOwnerRelationship"].get("isOfficer", False) == "1":
                print(transaction["reportingOwnerId"]["rptOwnerName"])
                #get code
                print(transaction['periodOfReport'])
                newDict[key] = value

print('Total: ',len(aggregates))
print('Filtered: ', len(newDict))
# print(newDict)

Bettinger Walter W
0001225208-21-004569
M
57815.0000
2021-03-01
Craig Jonathan M.
0001225208-19-010229
M
3740.0000
S
3740.0000
2019-07-15
Schwab Charles R.
0001225208-22-003462
G
24244.0000
2022-02-24
Murtagh Nigel J
0001225208-21-002664
M
19000.0000
M
17665.0000
S
36665.0000
2021-02-12
Morgan Peter J. III
0001225208-20-013174
S
3521.0000
2020-11-06
SCHWAB CHARLES R
0001225208-21-001688
M
126812.0000
S
6480.0000
S
17500.0000
S
126812.0000
M
147992.0000
S
147992.0000
2021-02-03
SCHWAB CHARLES R
0001225208-20-013984
S
98870.0000
2020-11-27
Craig Jonathan M.
0001225208-21-004752
S
6796.0000
2021-03-04
Schwab Charles R.
0001225208-22-001304
S
1420.0000
G
2778.0000
S
119280.0000
2022-02-01
Kallsen Terri R
0001225208-18-008015
M
18803.0000
M
17482.0000
S
17482.0000
S
18803.0000
2018-04-24
SCHWAB CHARLES R
0001225208-19-010425
G
85000.0000
2019-07-23
Morgan Peter J. III
0001225208-20-001429
M
6757.0000
M
6552.0000
M
3429.0000
S
3429.0000
S
6552.0000
S
6757.0000
2020-01-31
Schwab Charles R.
00

In [120]:
import numpy as np
from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook, reset_output
from bokeh.sampledata.stocks import AAPL, FB, GOOG, IBM, MSFT

# importing the modules
reset_output()
output_notebook()

# instantiating the figure object
graph = figure(x_axis_type = "datetime", title = "Stock Closing Prices")
graph.xaxis.axis_label = 'Date'
graph.yaxis.axis_label = 'Price (in USD)'
graph.legend.location = "top_left"

# plotting the line graph for GOOG
x_axis_coordinates = np.array(GOOG['date'], dtype = np.datetime64)
y_axis_coordinates = GOOG['adj_close']
legend_label = 'GOOG'
graph.line(x_axis_coordinates,
        y_axis_coordinates,
        legend_label = legend_label)

# displaying the model
show(graph)


Loading BokehJS ...

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/bokeh/models/plots.py:815: UserWarning: 
You are attempting to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
